In [4]:
%pip install selenium webdriver-manager

import requests
import pandas as pd
from datetime import datetime
import time

# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

def get_cme_settlements_robust(trade_date: str = None):
    """
    Scrapes CME settlement data using a Selenium-powered browser to bypass anti-bot protection.
    It initializes a browser to get valid session cookies, then uses requests to fetch the data.

    Args:
        trade_date (str, optional): Date in 'YYYYMMDD' format. Defaults to today.
    
    Returns:
        pandas.DataFrame: DataFrame with settlement data, or empty DataFrame on failure.
    """
    main_page_url = "https://www.cmegroup.com/markets/energy/refined-products/singapore-fob-marine-fuel-05-platts.settlements.html"
    api_url = "https://www.cmegroup.com/CmeWS/mvc/Settlements/Future/Settlements/4286/FUT"

    if not trade_date:
        trade_date = datetime.today().strftime('%Y%m%d')

    print("--- Step 1: Initializing automated browser to solve JS challenge ---")
    
    # Set up Chrome options for Selenium
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode (no browser window opens)
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    # Use webdriver-manager to automatically handle the driver
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
        
        # Navigate to the page and wait for JS to execute and set cookies
        driver.get(main_page_url)
        print("Waiting for page to load and security challenges to complete...")
        time.sleep(8)  # Wait 8 seconds. This is crucial for the JS to run.

        # Extract cookies from the browser session
        browser_cookies = driver.get_cookies()
        print("Successfully extracted browser cookies.")
        
    except Exception as e:
        print(f"An error occurred during the Selenium browser session: {e}")
        return pd.DataFrame()
    finally:
        # Ensure the browser is closed
        if 'driver' in locals():
            driver.quit()

    print("\n--- Step 2: Using extracted cookies to fetch data with requests ---")

    # Create a requests session
    session = requests.Session()

    # Add the cookies extracted from Selenium to our new session
    for cookie in browser_cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])

    # These headers are still important
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': main_page_url
    })

    params = {'tradeDate': trade_date, 'strategy': 'DEFAULT', 'pageSize': 500}
    
    try:
        print(f"Fetching data for trade date: {trade_date}...")
        response = session.get(api_url, params=params, timeout=15)
        response.raise_for_status()

        data = response.json()
        settlements_data = data.get('settlements', [])

        if not settlements_data:
            print(f"No settlement data found for {data.get('tradeDate', 'the specified date')}.")
            return pd.DataFrame()

        df = pd.DataFrame(settlements_data)
        print(f"Successfully retrieved data for trade date: {data.get('tradeDate')}")
        return df

    except requests.exceptions.RequestException as e:
        print(f"The final request to the API failed: {e}")
        return pd.DataFrame()


if __name__ == "__main__":
    print("--- Getting historical data for May 15, 2024 ---")
    historical_df = get_cme_settlements_robust(trade_date='20240515')
    if not historical_df.empty:
        print("\nHistorical Data for 2024-05-15:")
        print(historical_df[['month', 'settle', 'volume', 'openInterest']])

    print("\n" + "="*50 + "\n")

    print("--- Getting latest data ---")
    latest_df = get_cme_settlements_robust()
    if not latest_df.empty:
        print("\nLatest Data:")
        print(latest_df[['month', 'settle', 'volume', 'openInterest']])


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Z.Peng\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.4 MB 15.2 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/9.4 MB 13.0 MB/s eta 0:00:01
   ------ --------------------------------- 1.6/9.4 MB 12.5 MB/s eta 0:00:01
   --------- ------------------------------ 2.1/9.4 MB 12.3 MB/s eta 0:00:01
   ----------- ---------------------------- 2.7/9.4 MB 12.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.2/9.4 MB 12.1 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.4 MB 12.0 MB/s eta 0:00:01
   ------------------ --------------------- 4.3/9.4 MB 12.0 MB/s eta 0:00:01
   -------------------- ------------------- 4.9/9.4 MB 11.6 MB/s eta 0:00:01
   ----------------------- ---------------- 5.5/9.4 MB 11.6 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.4 MB 12.0 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.4 MB 12.0 MB/s eta 0:00:01
   ----